In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier


In [ ]:
X_train = pd.read_csv('/content/train.csv')
X_test = pd.read_csv('/content/test.csv')
sub = pd.read_csv('/content/gender_submission.csv')

In [ ]:
X_train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [ ]:
y_train = X_train['Survived']

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
X_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
X_train.drop(['Name', 'Cabin', 'Survived', 'Ticket' ], axis=1, inplace=True) 

In [ ]:
X_test.drop(['Name', 'Cabin', 'Ticket' ], axis=1, inplace=True) 

In [ ]:
X_train['Age'] = X_train['Age'].replace({'NAN':np.nan}).astype(float)
X_train["Age"] = X_train["Age"].fillna(value=X_train["Age"].mean())

In [ ]:
X_test['Age'] = X_test['Age'].replace({'NAN':np.nan}).astype(float)
X_test["Age"] = X_test["Age"].fillna(value=X_train["Age"].mean())

In [ ]:
X_test['Fare'] = X_test['Fare'].replace({'NAN':np.nan}).astype(float)
X_test["Fare"] = X_test["Fare"].fillna(value=X_train["Fare"].mean())


In [ ]:
le = preprocessing.LabelEncoder()
le.fit(X_train['Embarked'])
(X_train['Embarked'])= le.transform(X_train['Embarked'])

le = preprocessing.LabelEncoder()
le.fit(X_train['Sex'])
(X_train['Sex'])= le.transform(X_train['Sex'])

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(X_test['Embarked'])
(X_test['Embarked'])= le.transform(X_test['Embarked'])

le = preprocessing.LabelEncoder()
le.fit(X_test['Sex'])
(X_test['Sex'])= le.transform(X_test['Sex'])


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    int64  
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Embarked     891 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


In [ ]:
X_test.describe()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,0.636364,30.154603,0.447368,0.392344,35.619000,1.401914
std,120.810458,0.841838,0.481622,12.636666,0.896760,0.981429,55.840751,0.854496
min,892.000000,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,0.000000,23.000000,0.000000,0.000000,7.895800,1.000000
50%,1100.500000,3.000000,1.000000,29.699118,0.000000,0.000000,14.454200,2.000000
75%,1204.750000,3.000000,1.000000,35.750000,1.000000,0.000000,31.500000,2.000000
max,1309.000000,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


In [ ]:
y_train.describe()

count    891.000000
mean       0.383838
std        0.486592
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64

In [ ]:
X_train.dropna()
y_train.dropna()
X_test.dropna()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.500000,0,0,7.8292,1
1,893,3,0,47.000000,1,0,7.0000,2
2,894,2,1,62.000000,0,0,9.6875,1
3,895,3,1,27.000000,0,0,8.6625,2
4,896,3,0,22.000000,1,1,12.2875,2
...,...,...,...,...,...,...,...,...
413,1305,3,1,29.699118,0,0,8.0500,2
414,1306,1,0,39.000000,0,0,108.9000,0
415,1307,3,1,38.500000,0,0,7.2500,2
416,1308,3,1,29.699118,0,0,8.0500,2


In [ ]:
X_train.isna().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [ ]:
X_train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,1,22.000000,1,0,7.2500,2
1,2,1,0,38.000000,1,0,71.2833,0
2,3,3,0,26.000000,0,0,7.9250,2
3,4,1,0,35.000000,1,0,53.1000,2
4,5,3,1,35.000000,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
886,887,2,1,27.000000,0,0,13.0000,2
887,888,1,0,19.000000,0,0,30.0000,2
888,889,3,0,29.699118,1,2,23.4500,2
889,890,1,1,26.000000,0,0,30.0000,0


In [ ]:

rf = RandomForestClassifier(criterion='gini', 
                             n_estimators=100,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
model_Random= rf.fit(X_train, y_train)

pred_Random = model_Random.predict(X_test)



In [ ]:
#sub = pd.DataFrame(pred_Random, index =test.index)

In [ ]:
sub['Survived'] = pred_Random

In [ ]:
#sub.set_index(sub['PassengerId'])


In [ ]:
sub.to_csv('Submissãorandom.csv', index=False)

In [ ]:
import xgboost as xgb
my_model = xgb.XGBClassifier()
my_model.fit(X_train, y_train)
   
# Predicting the Test set results
y_pred = my_model.predict(X_test)

In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
sub['Survived'] = y_pred

In [ ]:
sub.to_csv('Submissãoxgboost.csv', index=False)

In [ ]:
sub['Survived'].value_counts()

0    287
1    131
Name: Survived, dtype: int64

In [ ]:
# Model

from sklearn.model_selection import RandomizedSearchCV

model = CatBoostClassifier(eval_metric='AUC',
                           random_seed = 1020,
                           bagging_temperature = 0.2,
                           od_type='Iter',
                           metric_period = 50,
                           od_wait=100)

parameters= {
               'learning_rate':[0.02],
               'max_depth': [15],
               'iterations': [300]
                }

clf = RandomizedSearchCV(model, parameters, random_state=0, cv=5, scoring='accuracy')

clf.fit(X_train, y_train, verbose=True)

preds = clf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


0:	total: 46.8ms	remaining: 14s
50:	total: 2.02s	remaining: 9.84s
100:	total: 7.63s	remaining: 15s
150:	total: 15.4s	remaining: 15.2s
200:	total: 23.1s	remaining: 11.4s
250:	total: 30.5s	remaining: 5.95s
299:	total: 38.1s	remaining: 0us
0:	total: 749us	remaining: 224ms
50:	total: 3.82s	remaining: 18.6s
100:	total: 9.92s	remaining: 19.5s
150:	total: 17.6s	remaining: 17.4s
200:	total: 25.1s	remaining: 12.4s
250:	total: 32.7s	remaining: 6.38s
299:	total: 41.3s	remaining: 0us
0:	total: 708us	remaining: 212ms
50:	total: 3.13s	remaining: 15.3s
100:	total: 13.7s	remaining: 26.9s
150:	total: 20.8s	remaining: 20.6s
200:	total: 28.6s	remaining: 14.1s
250:	total: 37.8s	remaining: 7.39s
299:	total: 47.2s	remaining: 0us
0:	total: 919us	remaining: 275ms
50:	total: 2.84s	remaining: 13.9s
100:	total: 8.02s	remaining: 15.8s
150:	total: 15.1s	remaining: 14.9s
200:	total: 22.7s	remaining: 11.2s
250:	total: 29.7s	remaining: 5.79s
299:	total: 38.6s	remaining: 0us
0:	total: 746us	remaining: 223ms
50:	total:

In [ ]:
preds

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
sub['Survived'] = preds

In [ ]:
sub.to_csv('Submissãoxgboost.csv', index=False)